In [ ]:
import numpy as np
import statsmodels.api as sm
import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage, BusinessDaysSincePreviousEvent, BusinessDaysUntilNextEvent
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline import factors, filters, classifiers
import alphalens
from scipy.stats.mstats import winsorize
from sklearn import preprocessing
from scipy.stats.mstats import gmean
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from quantopian.pipeline.data.factset import RBICSFocus
import quantopian.pipeline.data.factset.estimates as fe


In [ ]:
#Factors taken from https://www.quantopian.com/posts/alpha-combination-via-clustering
WIN_LIMIT = 0
QL = 66

def signalize(df):
    z = (df.rank() - 0.5)/df.count()
    return z.replace(np.nan, z.mean())

def preprocess(a):
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = winsorize(a, limits=[WIN_LIMIT,WIN_LIMIT])
    
    return preprocessing.scale(a)

class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 22  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)
        
class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1
    
    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]

class MessageSum(CustomFactor):
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, stocktwits.bull_scored_messages, stocktwits.bear_scored_messages, stocktwits.total_scanned_messages]
    window_length = 21
    window_safe = True
    def compute(self, today, assets, out, high, low, close, bull, bear, total):
        v = np.nansum((high-low)/close, axis=0)
        out[:] = preprocess(v*np.nansum(total*(bear-bull), axis=0))
                
class fcf(CustomFactor):
    inputs = [Fundamentals.fcf_yield]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, fcf_yield):
        out[:] = preprocess(np.nan_to_num(fcf_yield[-1,:]))

class FCFStability(CustomFactor):
    inputs = [Fundamentals.fcf_yield]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, fcf_yield):
        std = np.std([fcf_yield[-1], 
            fcf_yield[-QL], 
            fcf_yield[-QL*2],
            fcf_yield[-QL*3],
            fcf_yield[-QL*4],
            fcf_yield[-QL*5],
            fcf_yield[-QL*6],
            fcf_yield[-QL*7]], axis=0)

        out[:] = preprocess(-std)
        
class Direction(CustomFactor):
    inputs = [USEquityPricing.open, USEquityPricing.close]
    window_length = 21
    window_safe = True
    def compute(self, today, assets, out, open, close):
        p = (close-open)/close
        out[:] = preprocess(np.nansum(-p,axis=0))

class mean_rev(CustomFactor):   
    inputs = [USEquityPricing.high,USEquityPricing.low,USEquityPricing.close]
    window_length = 30
    window_safe = True
    def compute(self, today, assets, out, high, low, close):

        p = (high+low+close)/3

        m = len(close[0,:])
        n = len(close[:,0])
        b = np.zeros(m)
        a = np.zeros(m)

        for k in range(10,n+1):
            price_rel = np.nanmean(p[-k:,:],axis=0)/p[-1,:]
            wt = np.nansum(price_rel)
            b += wt*price_rel
            price_rel = 1.0/price_rel
            wt = np.nansum(price_rel)
            a += wt*price_rel

        out[:] = preprocess(b-a)

class volatility(CustomFactor):
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
    window_length = 5
    window_safe = True
    def compute(self, today, assets, out, high, low, close, volume):
        vol = np.nansum(volume,axis=0)*np.nansum(np.absolute((high-low)/close),axis=0)
        out[:] = preprocess(-vol)

class growthscore(CustomFactor):
    inputs = [Fundamentals.growth_score]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, growth_score):
        out[:] = preprocess(growth_score[-1,:])


class growthscorestability(CustomFactor):
    inputs = [Fundamentals.growth_score]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        out[:] = preprocess(-std)
                
class peg_ratio(CustomFactor):
    inputs = [Fundamentals.peg_ratio]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, peg_ratio):
        out[:] = preprocess(-1.0/peg_ratio[-1,:])

class peg_ratio_stability(CustomFactor):
    inputs = [Fundamentals.peg_ratio]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(std)
        
class MoneyflowVolume5d(CustomFactor):
    inputs = (USEquityPricing.close, USEquityPricing.volume)

    # we need one more day to get the direction of the price on the first
    # day of our desired window of 5 days
    window_length = 6
    window_safe = True

    def compute(self, today, assets, out, close_extra, volume_extra):
        # slice off the extra row used to get the direction of the close
        # on the first day
        close = close_extra[1:]
        volume = volume_extra[1:]

        dollar_volume = close * volume
        denominator = dollar_volume.sum(axis=0)

        difference = np.diff(close_extra, axis=0)
        direction = np.where(difference > 0, 1, -1)
        numerator = (direction * dollar_volume).sum(axis=0)

        out[:] = preprocess(-np.divide(numerator, denominator))
        
class Trendline(CustomFactor):
        inputs = [USEquityPricing.close]
        window_length = 252
        window_safe = True
            
        _x = np.arange(window_length)
        _x_var = np.var(_x)
 
        def compute(self, today, assets, out, close):
            
            x_matrix = repeat_last_axis(
            (self.window_length - 1) / 2 - self._x,
            len(assets),
            )
 
            y_bar = np.nanmean(close, axis=0)
            y_bars = repeat_first_axis(y_bar, self.window_length)
            y_matrix = close - y_bars
 
            out[:] = preprocess(-np.divide(
            (x_matrix * y_matrix).sum(axis=0) / self._x_var,
            self.window_length
            ))
                
class SalesGrowth(CustomFactor):
    inputs = [factset.Fundamentals.sales_gr_qf]
    window_length = 2*252
    window_safe = True
    def compute(self, today, assets, out, sales_growth):
        sales_growth = np.nan_to_num(sales_growth)
        sales_growth = preprocessing.scale(sales_growth,axis=0)
        out[:] = preprocess(sales_growth[-1])

class SalesGrowthStability(CustomFactor):
    inputs = [factset.Fundamentals.sales_gr_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        mean = np.mean([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(mean/std)

        
class GrossMarginChange(CustomFactor):
    window_length = 2*252
    window_safe = True
    inputs = [factset.Fundamentals.ebit_oper_mgn_qf]
    def compute(self, today, assets, out, ebit_oper_mgn):
        ebit_oper_mgn = np.nan_to_num(ebit_oper_mgn)
        ebit_oper_mgn = preprocessing.scale(ebit_oper_mgn,axis=0)
        out[:] = preprocess(ebit_oper_mgn[-1])

class GrossMarginStability(CustomFactor):
    window_length = QL*8
    window_safe = True
    inputs = [factset.Fundamentals.ebit_oper_mgn_qf]
    def compute(self, today, assets, out, var):
        var = np.nan_to_num(var)
        mean = np.mean([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(mean/std)

        
class Gross_Income_Margin(CustomFactor):
    #Gross Income Margin:
    #Gross Profit divided by Net Sales
    #Notes:
    #High value suggests that the company is generating large profits
    inputs = [Fundamentals.cost_of_revenue, Fundamentals.total_revenue]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, cost_of_revenue, sales):
        gross_income_margin = sales[-1]/sales[-1] - cost_of_revenue[-1]/sales[-1]
        out[:] = preprocess(-gross_income_margin)


class Gross_Income_Margin_Stability(CustomFactor):
    #Gross Income Margin:
    #Gross Profit divided by Net Sales
    #Notes:
    #High value suggests that the company is generating large profits
    inputs = [Fundamentals.cost_of_revenue, Fundamentals.total_revenue]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, cost_of_revenue, sales):
        var = 1 - cost_of_revenue/sales
        mean = np.mean([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(mean/std)
        
class MaxGap(CustomFactor): 
    # the biggest absolute overnight gap in the previous 90 sessions
    inputs = [USEquityPricing.close] ; window_length = 90
    window_safe = True
    def compute(self, today, assets, out, close):
        abs_log_rets = np.abs(np.diff(np.log(close),axis=0))
        max_gap = np.max(abs_log_rets, axis=0)
        out[:] = preprocess(max_gap)

class CapEx_Vol(CustomFactor):
    inputs=[
        factset.Fundamentals.capex_assets_qf]
    window_length = 2*252
    window_safe = True
    def compute(self, today, assets, out, capex_assets):
        
        out[:] = preprocess(-np.ptp(capex_assets,axis=0))

class CapEx_Vol_Stability(CustomFactor):
    inputs=[
        factset.Fundamentals.capex_assets_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(-std)

        
class fcf_ev(CustomFactor):
    inputs=[
        Fundamentals.fcf_per_share,
        Fundamentals.shares_outstanding,
        Fundamentals.enterprise_value,]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, fcf, shares, ev):
        v = fcf*shares/ev
        v[np.isinf(v)] = np.nan

        out[:] = preprocess(v[-1])

        
class fcf_ev_stability(CustomFactor):
    inputs=[
        Fundamentals.fcf_per_share,
        Fundamentals.shares_outstanding,
        Fundamentals.enterprise_value,]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, fcf, shares, ev):
        var = fcf*shares/ev
        var[np.isinf(var)] = np.nan
        mean = np.mean([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(mean)


class fcf_growth_mean(CustomFactor):
    inputs=[
        Fundamentals.fcf_per_share,
        Fundamentals.shares_outstanding,
        Fundamentals.enterprise_value,]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, fcf, shares, ev):
        var = fcf*shares
        var[np.isinf(var)] = np.nan
        
        arr = [var[-1]/var[-QL] -1, 
            var[-QL]/var[-2*QL] -1,
            var[-QL*2]/var[-3*QL] -1, 
            var[-QL*3]/var[-4*QL] -1, 
            var[-QL*4]/var[-5*QL] -1, 
            var[-QL*5]/var[-6*QL] -1, 
            var[-QL*6]/var[-7*QL] -1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
                
        
        
class DebtToTotalAssets(CustomFactor):
    inputs = [Fundamentals.long_term_debt,
        Fundamentals.current_debt,
        Fundamentals.cash_and_cash_equivalents,
        Fundamentals.total_assets]
    window_length = 1
    window_safe = True

    def compute(self, today, assets, out, ltd, std, cce, ta):
        std_part = np.maximum(std - cce, np.zeros(std.shape))
        v = np.divide(ltd + std_part, ta)
        v[np.isinf(v)] = np.nan
        out[:] = preprocess(np.ravel(v))

class DebtToTotalAssetsStability(CustomFactor):
    inputs = [Fundamentals.long_term_debt,
        Fundamentals.current_debt,
        Fundamentals.cash_and_cash_equivalents,
        Fundamentals.total_assets]
    window_length = QL*8
    window_safe = True

    def compute(self, today, assets, out, ltd, std, cce, ta):
        std_part = np.maximum(std - cce, np.zeros(std.shape))
        var = np.divide(ltd + std_part, ta)
        var[np.isinf(var)] = np.nan
        mean = np.mean([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(std)


class dta_growth_mean(CustomFactor):
    inputs = [Fundamentals.long_term_debt,
        Fundamentals.current_debt,
        Fundamentals.cash_and_cash_equivalents,
        Fundamentals.total_assets]
    window_length = QL*8
    window_safe = True
    
    def compute(self, today, assets, out, ltd, std, cce, ta):
        std_part = np.maximum(std - cce, np.zeros(std.shape))
        var = np.divide(ltd + std_part, ta)
        var[np.isinf(var)] = np.nan
     
        arr = [var[-1]/var[-QL] -1, 
            var[-QL]/var[-2*QL] -1,
            var[-QL*2]/var[-3*QL] -1, 
            var[-QL*3]/var[-4*QL] -1, 
            var[-QL*4]/var[-5*QL] -1, 
            var[-QL*5]/var[-6*QL] -1, 
            var[-QL*6]/var[-7*QL] -1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-mean)
        
        
        
class TEM(CustomFactor):
    """
    TEM = standard deviation of past 6 quarters' reports
    """
    inputs=[factset.Fundamentals.capex_qf_asof_date,
        factset.Fundamentals.capex_qf,
        factset.Fundamentals.assets]
    window_length = 390
    window_safe = True
    def compute(self, today, assets, out, asof_date, capex, total_assets):
        values = capex/total_assets
        values[np.isinf(values)] = np.nan
        out_temp = np.zeros_like(values[-1,:])
        for column_ix in range(asof_date.shape[1]):
            _, unique_indices = np.unique(asof_date[:, column_ix], return_index=True)
            quarterly_values = values[unique_indices, column_ix]
            if len(quarterly_values) < 6:
                quarterly_values = np.hstack([
                np.repeat([np.nan], 6 - len(quarterly_values)),
                quarterly_values,
                ])

            out_temp[column_ix] = np.std(quarterly_values[-6:])

        out[:] = preprocess(-out_temp)

class TEM_GROWTH(CustomFactor):
    """
    TEM = standard deviation of past 6 quarters' reports
    """
    inputs=[factset.Fundamentals.capex_qf_asof_date,
        factset.Fundamentals.capex_qf,
        factset.Fundamentals.assets]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, asof_date, capex, total_assets):
        var = capex/total_assets
        var[np.isinf(var)] = np.nan
        arr = [var[-1]/var[-QL] -1, 
            var[-QL]/var[-2*QL] -1,
            var[-QL*2]/var[-3*QL] -1, 
            var[-QL*3]/var[-4*QL] -1, 
            var[-QL*4]/var[-5*QL] -1, 
            var[-QL*5]/var[-6*QL] -1, 
            var[-QL*6]/var[-7*QL] -1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
        
class Piotroski(CustomFactor):
    inputs = [
            Fundamentals.roa,
            Fundamentals.operating_cash_flow,
            Fundamentals.cash_flow_from_continuing_operating_activities,
            Fundamentals.long_term_debt_equity_ratio,
            Fundamentals.current_ratio,
            Fundamentals.shares_outstanding,
            Fundamentals.gross_margin,
            Fundamentals.assets_turnover,
            ]

    window_length = 100
    window_safe = True

    def compute(self, today, assets, out,roa, cash_flow, cash_flow_from_ops, long_term_debt_ratio, current_ratio, shares_outstanding, gross_margin, assets_turnover):

        profit = (
                    (roa[-1] > 0).astype(int) +
                    (cash_flow[-1] > 0).astype(int) +
                    (roa[-1] > roa[0]).astype(int) +
                    (cash_flow_from_ops[-1] > roa[-1]).astype(int)
                )

        leverage = (
                    (long_term_debt_ratio[-1] < long_term_debt_ratio[0]).astype(int) +
                    (current_ratio[-1] > current_ratio[0]).astype(int) + 
                    (shares_outstanding[-1] <= shares_outstanding[0]).astype(int)
                    )

        operating = (
                    (gross_margin[-1] > gross_margin[0]).astype(int) +
                    (assets_turnover[-1] > assets_turnover[0]).astype(int)
                    )

        out[:] = preprocess(profit + leverage + operating)

class Altman_Z(CustomFactor):
    inputs=[factset.Fundamentals.zscore_qf]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, zscore_qf):
        out[:] = preprocess(zscore_qf[-1])

        
class Altman_Z_Stability(CustomFactor):
    inputs=[factset.Fundamentals.zscore_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-mean/std)
        
        
class Quick_Ratio(CustomFactor):
    inputs=[factset.Fundamentals.quick_ratio_qf]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, quick_ratio_qf):
        out[:] = preprocess(quick_ratio_qf[-1])
    

class Quick_Ratio_Std(CustomFactor):
    inputs=[factset.Fundamentals.quick_ratio_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
    
class Quick_Ratio_Mean(CustomFactor):
    inputs=[factset.Fundamentals.quick_ratio_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)

class Quick_Ratio_Stability(CustomFactor):
    inputs=[factset.Fundamentals.quick_ratio_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
    
class AdvancedMomentum(CustomFactor):
    inputs = (USEquityPricing.close, Returns(window_length=126))
    window_length = 252
    window_safe = True

    def compute(self, today, assets, out, prices, returns):
        am = np.divide(
        (
        (prices[-21] - prices[-252]) / prices[-252] -
        prices[-1] - prices[-21]
        ) / prices[-21],
        np.nanstd(returns, axis=0)
        )

        out[:] = preprocess(-am)

class STA(CustomFactor):  
    inputs = [Fundamentals.operating_cash_flow,  
              Fundamentals.net_income_continuous_operations,  
              Fundamentals.total_assets]  
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, ocf, ni, ta):  
        ta = np.where(np.isnan(ta), 0, ta)  
        ocf = np.where(np.isnan(ocf), 0, ocf)  
        ni = np.where(np.isnan(ni), 0, ni)  
        out[:] = preprocess(abs(ni[-1] - ocf[-1])/ ta[-1])
        
class STA_Stability(CustomFactor):  
    inputs = [Fundamentals.operating_cash_flow,  
              Fundamentals.net_income_continuous_operations,  
              Fundamentals.total_assets]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, ocf, ni, ta):  
        ta = np.where(np.isnan(ta), 0, ta)  
        ocf = np.where(np.isnan(ocf), 0, ocf)  
        ni = np.where(np.isnan(ni), 0, ni)  
        var = abs(ni - ocf)/ ta
        
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)

class STA_Mean(CustomFactor):  
    inputs = [Fundamentals.operating_cash_flow,  
              Fundamentals.net_income_continuous_operations,  
              Fundamentals.total_assets]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, ocf, ni, ta):  
        ta = np.where(np.isnan(ta), 0, ta)  
        ocf = np.where(np.isnan(ocf), 0, ocf)  
        ni = np.where(np.isnan(ni), 0, ni)  
        var = abs(ni - ocf)/ ta
        
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
        
class STA_Std(CustomFactor):  
    inputs = [Fundamentals.operating_cash_flow,  
              Fundamentals.net_income_continuous_operations,  
              Fundamentals.total_assets]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, ocf, ni, ta):  
        ta = np.where(np.isnan(ta), 0, ta)  
        ocf = np.where(np.isnan(ocf), 0, ocf)  
        ni = np.where(np.isnan(ni), 0, ni)  
        var = abs(ni - ocf)/ ta
        
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
        
class SNOA(CustomFactor):  
    inputs = [Fundamentals.total_assets,  
             Fundamentals.cash_and_cash_equivalents,  
             Fundamentals.current_debt, # same as short-term debt?  
             Fundamentals.minority_interest_balance_sheet,  
             Fundamentals.long_term_debt, # check same?  
             Fundamentals.preferred_stock] # check same?  
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, ta, cace, cd, mi, ltd, ps):  
        ta = np.where(np.isnan(ta), 0, ta)  
        cace = np.where(np.isnan(cace), 0, cace)  
        cd = np.where(np.isnan(cd), 0, cd)  
        mi = np.where(np.isnan(mi), 0, mi)  
        ltd = np.where(np.isnan(ltd), 0, ltd)  
        ps = np.where(np.isnan(ps), 0, ps)  
        results = ((ta[-1]-cace[-1])-(ta[-1]-cace[-1]-ltd[-1]-cd[-1]-ps[-1]-mi[-1]))/ta[-1]  
        out[:] = preprocess(np.where(np.isnan(results),0,results))

class ROA(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, roa):  
        out[:] = preprocess(np.where(roa[-1]>0,1,0))

        
class ROA_Mean(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1]/var[-QL] - 1,
            var[-QL]/var[-2*QL] - 1,
            var[-QL*2]/var[-3*QL] - 1,
            var[-QL*3]/var[-4*QL] - 1,
            var[-QL*4]/var[-5*QL] - 1,
            var[-QL*5]/var[-6*QL] - 1,
            var[-QL*6]/var[-7*QL] - 1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)

class ROA_Std(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1]/var[-QL] - 1,
            var[-QL]/var[-2*QL] - 1,
            var[-QL*2]/var[-3*QL] - 1,
            var[-QL*3]/var[-4*QL] - 1,
            var[-QL*4]/var[-5*QL] - 1,
            var[-QL*5]/var[-6*QL] - 1,
            var[-QL*6]/var[-7*QL] - 1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)

class ROA_Stability(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1]/var[-QL] - 1,
            var[-QL]/var[-2*QL] - 1,
            var[-QL*2]/var[-3*QL] - 1,
            var[-QL*3]/var[-4*QL] - 1,
            var[-QL*4]/var[-5*QL] - 1,
            var[-QL*5]/var[-6*QL] - 1,
            var[-QL*6]/var[-7*QL] - 1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class FCFTA(CustomFactor):  
    inputs = [Fundamentals.free_cash_flow,  
             Fundamentals.total_assets]  
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, fcf, ta):  
        out[:] = preprocess(np.where(fcf[-1]/ta[-1]>0,1,0))

class ROA_GROWTH(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, roa):  
        out[:] = np.where(roa[-1]>roa[-252],1,0)

class FCFTA_ROA(CustomFactor):  
    inputs = [Fundamentals.free_cash_flow,  
              Fundamentals.total_assets,  
              Fundamentals.roa]  
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, fcf, ta, roa):  
        out[:] = preprocess(np.where(fcf[-1]/ta[-1]>roa[-1],1,0))

class FCFTA_GROWTH(CustomFactor):  
    inputs = [Fundamentals.free_cash_flow,  
              Fundamentals.total_assets]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, fcf, ta):  
        out[:] = preprocess(np.where(fcf[-1]/ta[-1]>fcf[-252]/ta[-252],1,0))

class LTD_GROWTH(CustomFactor):  
    inputs = [Fundamentals.total_assets,  
              Fundamentals.long_term_debt]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, ta, ltd):  
        out[:] = preprocess(np.where(ltd[-1]/ta[-1]<ltd[-252]/ta[-252],1,0))

class CR_GROWTH(CustomFactor):  
    inputs = [Fundamentals.current_ratio]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, cr):  
        out[:] = preprocess(np.where(cr[-1]>cr[-252],1,0))
        
class Current_Ratio_Std(CustomFactor):
    inputs=[Fundamentals.current_ratio]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
    
class Current_Ratio_Mean(CustomFactor):
    inputs=[Fundamentals.current_ratio]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)

class Current_Ratio_Stability(CustomFactor):
    inputs=[Fundamentals.current_ratio]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        

class GM_GROWTH(CustomFactor):  
    inputs = [Fundamentals.gross_margin]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, gm):  
        out[:] = preprocess(np.where(gm[-1]>gm[-252],1,0))

class ATR_GROWTH(CustomFactor):  
    inputs = [Fundamentals.assets_turnover]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, atr):  
        out[:] = preprocess(np.where(atr[-1]>atr[-252],1,0))

class ATR_GROWTH_MEAN(CustomFactor):  
    inputs = [Fundamentals.assets_turnover]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
    

class ATR_GROWTH_STD(CustomFactor):  
    inputs = [Fundamentals.assets_turnover]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
        
class ATR_GROWTH_STABILITY(CustomFactor):  
    inputs = [Fundamentals.assets_turnover]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class NEQISS(CustomFactor):  
    inputs = [Fundamentals.shares_outstanding]  
    window_length = 252
    window_safe = True
    def compute(self, today, assets, out, so):  
        out[:] = preprocess(np.where(so[-1]-so[-252]<1,1,0))

class GM_GROWTH_2YR(CustomFactor):  
    inputs = [Fundamentals.gross_margin]  
    window_length = 504
    window_safe = True
    def compute(self, today, assets, out, gm):  
        out[:] = preprocess(gmean([gm[-1]+1, gm[-252]+1,gm[-504]+1])-1)

class GM_STABILITY_2YR(CustomFactor):  
    inputs = [Fundamentals.gross_margin]  
    window_length = 504
    window_safe = True
    def compute(self, today, assets, out, gm):  
        out[:] = preprocess(-np.std([gm[-1]-gm[-252],gm[-252]-gm[-504]],axis=0)) 

class ROA_GROWTH_2YR(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = 504
    window_safe = True
    def compute(self, today, assets, out, roa):  
        out[:] = preprocess(gmean([roa[-1]+1, roa[-252]+1,roa[-504]+1])-1)

class ROIC_GROWTH_2YR(CustomFactor):  
    inputs = [Fundamentals.roic]  
    window_length = 504
    window_safe = True
    def compute(self, today, assets, out, roic):  
        out[:] = preprocess(gmean([roic[-1]+1, roic[-252]+1,roic[-504]+1])-1)

class GM_GROWTH_8YR(CustomFactor):  
    inputs = [Fundamentals.gross_margin]  
    window_length = 8
    window_safe = True
    def compute(self, today, assets, out, gm):  
        out[:] = preprocess(gmean([gm[-1]+1, gm[-2]+1, gm[-3]+1, gm[-4]+1, gm[-5]+1, gm[-6]+1, gm[-7]+1, gm[-8]+1])-1)        

class GM_STABILITY_8YR(CustomFactor):  
    inputs = [Fundamentals.gross_margin]  
    window_length = 9
    window_safe = True
    def compute(self, today, assets, out, gm):  
        out[:] = preprocess(gm[-8]) 

class ROA_GROWTH_8YR(CustomFactor):  
    inputs = [Fundamentals.roa]  
    window_length = 9
    window_safe = True
    def compute(self, today, assets, out, roa):  
        out[:] = preprocess(gmean([roa[-1]/100+1, roa[-2]/100+1,roa[-3]/100+1,roa[-4]/100+1,roa[-5]/100+1,roa[-6]/100+1,roa[-7]/100+1,roa[-8]/100+1])-1) 

class ROIC_GROWTH_8YR(CustomFactor):  
    inputs = [Fundamentals.roic]  
    window_length = 9
    window_safe = True
    def compute(self, today, assets, out, roic):  
        out[:] = preprocess(gmean([roic[-1]/100+1, roic[-2]/100+1,roic[-3]/100+1,roic[-4]/100+1,roic[-5]/100+1,roic[-6]/100+1,roic[-7]/100+1,roic[-8]/100+1])-1)

class Quick_Ratio_Stability(CustomFactor):
    inputs = [Fundamentals.quick_ratio] 
    window_length = 360
    def compute(self, today, assets, out, rate):
        out[:] = preprocess(rate.mean(axis = 0)/rate.std(axis = 0))        
        

class pcf_ratio_stability(CustomFactor):
    inputs = [Fundamentals.pcf_ratio]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(std)
        
class ps_ratio_stability(CustomFactor):
    inputs = [Fundamentals.ps_ratio]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(std)

class fey_stability(CustomFactor):
    inputs = [Fundamentals.forward_earning_yield]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(std)

class ey_stability(CustomFactor):
    inputs = [Fundamentals.earning_yield]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(-std)

class sy_stability(CustomFactor):
    inputs = [Fundamentals.sales_yield]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(-std)
        
class revenue_growth_mean(CustomFactor):  
    inputs = [Fundamentals.revenue_growth]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)

class revenue_growth_std(CustomFactor):  
    inputs = [Fundamentals.revenue_growth]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)

class revenue_growth_stability(CustomFactor):  
    inputs = [Fundamentals.revenue_growth]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)

class ebitda_margin_mean(CustomFactor):  
    inputs = [Fundamentals.ebitda_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
        
class ebitda_margin_std(CustomFactor):  
    inputs = [Fundamentals.ebitda_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
        
class ebitda_margin_stability(CustomFactor):  
    inputs = [Fundamentals.ebitda_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)

class pretax_margin_mean(CustomFactor):  
    inputs = [Fundamentals.pretax_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
        
class pretax_margin_std(CustomFactor):  
    inputs = [Fundamentals.pretax_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
        
class pretax_margin_stability(CustomFactor):  
    inputs = [Fundamentals.pretax_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        

class trading_profit_stability(CustomFactor):  
    inputs = [Fundamentals.trading_gain_loss]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class trading_profit_mean(CustomFactor):  
    inputs = [Fundamentals.trading_gain_loss]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
        
        
sector = RBICSFocus.l1_name.latest
base_universe = QTradableStocksUS() & sector.eq('Finance')

guidance_days_prev = BusinessDaysSincePreviousEvent(inputs=[fe.Actuals.slice('SALES', 'qf', 0).asof_date])  
guidance_days_next = BusinessDaysUntilNextEvent(inputs=[fe.Actuals.slice('SALES', 'qf', 0).asof_date])  

mkt_cap = MarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

alpha1 = MessageSum(mask = base_universe)        
alpha2 = fcf(mask = base_universe)        
alpha3 = Direction(mask = base_universe)        
alpha4 = mean_rev(mask = base_universe)        
alpha5 = volatility(mask = base_universe)        
alpha6 = growthscore(mask = base_universe)        
alpha7 = peg_ratio(mask = base_universe)        
alpha8 = MoneyflowVolume5d(mask = base_universe)        
alpha9 = Trendline(mask = base_universe)        
alpha10 = SalesGrowth(mask = base_universe)        
alpha11 = GrossMarginChange(mask = base_universe)        
alpha12 = Gross_Income_Margin(mask = base_universe)        
alpha13 = MaxGap(mask = base_universe)        
alpha14 = CapEx_Vol(mask = base_universe)        
alpha15 = fcf_ev(mask = base_universe)        
alpha16 = DebtToTotalAssets(mask = base_universe)        
alpha17 = TEM(mask = base_universe)        
alpha18 = Piotroski(mask = base_universe)        
alpha19 = Altman_Z(mask = base_universe)        
alpha20 = Quick_Ratio(mask = base_universe)        
alpha21 = AdvancedMomentum(mask = base_universe)        
alpha22 = STA(mask = base_universe)        
alpha23 = SNOA(mask = base_universe)        
alpha24 = ROA(mask = base_universe)        
alpha25 = FCFTA(mask = base_universe)        
alpha26 = ROA_GROWTH(mask = base_universe)        
alpha27 = FCFTA_ROA(mask = base_universe)        
alpha28 = FCFTA_GROWTH(mask = base_universe)        
alpha29 = LTD_GROWTH(mask = base_universe)        
alpha30 = CR_GROWTH(mask = base_universe)        
alpha31 = GM_GROWTH(mask = base_universe) 
alpha32 = ATR_GROWTH(mask = base_universe) 
alpha33 = NEQISS(mask = base_universe) 
alpha34 = GM_GROWTH_2YR(mask = base_universe) 
alpha35 = GM_STABILITY_2YR(mask = base_universe) 
alpha36 = ROA_GROWTH_2YR(mask = base_universe) 
alpha37 = ROIC_GROWTH_2YR(mask = base_universe) 
alpha38 = GM_GROWTH_8YR(mask = base_universe) 
alpha39 = GM_STABILITY_8YR(mask = base_universe) 
alpha40 = ROA_GROWTH_8YR(mask = base_universe) 
alpha41 = ROIC_GROWTH_8YR(mask = base_universe) 
alpha42 = Quick_Ratio_Stability(mask = base_universe)
alpha43 = FCFStability(mask = base_universe)
alpha44 = growthscorestability(mask = base_universe)
alpha45 = peg_ratio_stability(mask = base_universe)
alpha46 = SalesGrowthStability(mask = base_universe)
alpha47 = GrossMarginStability(mask = base_universe)
alpha48 = Gross_Income_Margin_Stability(mask = base_universe)
alpha49 = CapEx_Vol_Stability(mask = base_universe)
alpha50 = fcf_ev_stability(mask = base_universe)
alpha51 = DebtToTotalAssetsStability(mask = base_universe)
alpha52 = fcf_growth_mean(mask = base_universe)
alpha53 = dta_growth_mean(mask = base_universe)
alpha54 = TEM_GROWTH(mask = base_universe)
alpha55 = Altman_Z_Stability(mask = base_universe)
alpha56 = Quick_Ratio_Mean(mask = base_universe)
alpha57 = Quick_Ratio_Std(mask = base_universe)
alpha58 = Quick_Ratio_Stability(mask = base_universe)
alpha59 = STA_Stability(mask = base_universe)
alpha60 = STA_Mean(mask = base_universe)
alpha61 = STA_Std(mask = base_universe)
alpha62 = ROA_Mean(mask = base_universe)
alpha63 = ROA_Std(mask = base_universe)
alpha64 = ROA_Stability(mask = base_universe)
alpha65 = Current_Ratio_Mean(mask = base_universe)
alphaQL = Current_Ratio_Std(mask = base_universe)
alpha67 = Current_Ratio_Stability(mask = base_universe)
alpha68 = ATR_GROWTH_MEAN(mask = base_universe)
alpha69 = ATR_GROWTH_STD(mask = base_universe)
alpha70 = ATR_GROWTH_STABILITY(mask = base_universe)
alpha71 = pcf_ratio_stability(mask = base_universe)
alpha72 = ps_ratio_stability(mask = base_universe)
alpha73 = fey_stability(mask = base_universe)
alpha74 = ey_stability(mask = base_universe)
alpha75 = sy_stability(mask = base_universe)
alpha76 = revenue_growth_mean(mask = base_universe)
alpha77 = revenue_growth_std(mask = base_universe)
alpha78 = revenue_growth_stability(mask = base_universe)
alpha79 = ebitda_margin_mean(mask = base_universe)
alpha80 = ebitda_margin_std(mask = base_universe)
alpha81 = ebitda_margin_stability(mask = base_universe)
alpha82 = pretax_margin_mean(mask = base_universe)
alpha83 = pretax_margin_std(mask = base_universe)
alpha84 = pretax_margin_stability(mask = base_universe)

alpha85 = trading_profit_stability(mask = base_universe)
alpha86 = trading_profit_mean(mask = base_universe)


pipe_alpha_factors = Pipeline(
    columns={
'vol': vol,
'mkt_cap':mkt_cap,
        
# 'alpha1':alpha1,
# 'alpha2':alpha2,
# 'alpha3':alpha3,

        'alpha4':alpha4,

# 'alpha5':alpha5,
# 'alpha6':alpha6,
# 'alpha7':alpha7,
# 'alpha8':alpha8,
# 'alpha9':alpha9,
# 'alpha10':alpha10,
# 'alpha11':alpha11,

        'alpha12':alpha12,
        'alpha13':alpha13,
        
# 'alpha14':alpha14,
# 'alpha15':alpha15,
# 'alpha16':alpha16,
# 'alpha17':alpha17,
# 'alpha18':alpha18,
# 'alpha19':alpha19,
# 'alpha20':alpha20,
# 'alpha21':alpha21,
# 'alpha22':alpha22,
# 'alpha23':alpha23,
# 'alpha24':alpha24,

        'alpha25':alpha25,

# 'alpha26':alpha26,
# 'alpha27':alpha27,
# 'alpha28':alpha28,
# 'alpha29':alpha29,
# 'alpha30':alpha30,
# 'alpha31':alpha31,
# 'alpha32':alpha32,
# 'alpha33':alpha33,
# 'alpha34':alpha34,
# 'alpha35':alpha35,
# 'alpha36':alpha36,
# 'alpha37':alpha37,
# 'alpha38':alpha38,
# 'alpha39':alpha39,
# 'alpha40':alpha40,
# 'alpha41':alpha41,
# 'alpha42': alpha42,

        'alpha43': alpha43,
        'alpha44':alpha44,
        
# 'alpha45':alpha45,

        'alpha46': alpha46,

# 'alpha47': alpha47,
# 'alpha48': alpha48,
# 'alpha49': alpha49,S
# 'alpha50': alpha50,
# 'alpha51': alpha51,
        
        'alpha52': alpha52,

# 'alpha53': alpha53,
        'alpha54': alpha54,
# 'alpha55': alpha55,
# 'alpha56': alpha56,
# 'alpha57': alpha57,
# 'alpha58': alpha58,
        'alpha59': alpha59,

# 'alpha60': alpha60,
# 'alpha61': alpha61,
# 'alpha62':alpha62,
# 'alpha63':alpha63,
# 'alpha64':alpha64,
        'alpha65':alpha65,

# 'alphaQL':alphaQL,
# 'alpha67':alpha67,
# 'alpha68':alpha68,
# 'alpha69':alpha69,
# 'alpha70':alpha70,
        'alpha71':alpha71,
        'alpha72':alpha72,
# 'alpha73':alpha73,
# 'alpha74':alpha74,
# 'alpha75':alpha75,
        'alpha76': alpha76,

# 'alpha77': alpha77,
        'alpha78': alpha78,

# 'alpha79': alpha79,
# 'alpha80': alpha80,
# 'alpha81': alpha81,

# 'alpha82': alpha82,
# 'alpha83': alpha83,
        'alpha84': alpha84,
        
        'alpha85': alpha85,
        'alpha86': alpha86,
        
        'guidance_days_next': guidance_days_next,
        'guidance_days_prev': guidance_days_prev
    },
    screen=base_universe
)   

output = run_pipeline(pipe_alpha_factors, '2015-06-01', '2018-09-01')

In [ ]:
assets = output.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
pricing = get_pricing(assets, start_date='2015-06-01', end_date='2018-09-01', fields='close_price')


In [ ]:
# factor_names = ['alpha1','alpha2','alpha3','alpha4','alpha5','alpha6','alpha7','alpha8','alpha9','alpha10',
# 'alpha11','alpha12','alpha13','alpha14','alpha15','alpha16','alpha17','alpha18','alpha19','alpha20',
# 'alpha21','alpha22','alpha23','alpha24','alpha25','alpha26','alpha27','alpha28','alpha29','alpha30',
# 'alpha31','alpha32','alpha33','alpha34','alpha35','alpha36','alpha37','alpha38','alpha39','alpha40',
# 'alpha41']

# factor_names = ['alpha2','alpha4','alpha11','alpha14','alpha31',
#                 'alpha35', 'alpha43', 'alpha45', 'alpha46', 'alpha52', 
#                 'alpha55', 'alpha66', 'alpha73', 'alpha77']
# # 'alpha82', 'alpha83', 'alpha84']

factor_names = ['alpha4', 'alpha43', 'alpha46', 'alpha52', 
                'alpha12', 'alpha13', 'alpha25', 'alpha44', 
                  'alpha46', 'alpha54', 'alpha59', 'alpha65',
                'alpha71', 'alpha72', 'alpha76', 'alpha78', 'alpha84',
                'alpha85', 'alpha86'
               ]

# for i in range(53,85):
#     if i not in [4,43,46,52,12,13,25,44]:
#         factor_names.append('alpha'+str(i))

def wmean(name):
    return (output['mkt_cap']*output[name]).sum(level=0)/output['mkt_cap'].mean(level=0)

# data = pd.DataFrame()
data = output[['guidance_days_prev','guidance_days_next']]

#Demean the alphas
for name in factor_names:
    output['s'+name] = output[name]
    data[name] = output['s'+name].sub(output['s'+name].mean(level=0), level=0)
#     output[name] = output[name].sub(wmean(name), level=0)



In [ ]:
# output['vol_rank'] = signalize(output['vol'])
# output['mk_rank'] = signalize(output['mkt_cap']**(1.0/3.0))
# # output['weights'] = 0.3*output['vol_rank'] + 0.7*output['mk_rank']
# # output['weights'] = output['vol_rank'] 
# output['weights'] = output['vol_rank']/output['vol_rank'].sum()

In [ ]:
returns = pricing.pct_change(periods=5).shift(-5)
#Demean the returns
returns = returns.sub(returns.mean(axis=1), axis=0).stack()
data = pd.concat([returns.rename('returns'), data], axis=1).dropna()

In [ ]:
data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})

In [ ]:
# data['d1'] = 0
# data.loc[(data['guidance_days_next'] > -10) | (data['guidance_days_prev'] < 10), 'd1'] = 1.0
# print data.loc[data['guidance_days_prev'] < 10, 'guidance_days_prev']
# print data['guidance_days_prev']

In [ ]:
# print data.head(10)

In [ ]:
# significant_factor_names = ['alpha2', 'alpha4', 'alpha11', 'alpha14', 
#                             'alpha31', 'alpha35', 'alpha43', 'alpha45', 'alpha46', 
#                             'alpha52', 'alpha55', 
#                             'alpha66', 
#                             'alpha73',
#                             'alpha77'
#                              ]

# data['alpha_k1'] = data['alpha10'] + data['alpha12'] + data['alpha13'] + data['alpha25'] + data['alpha27'] 
# data['alpha_k2'] = data['alpha17'] + data['alpha18'] + data['alpha19'] 

# data['alpha_s1'] = -(data['alpha9'] + data['alpha16'] + data['alpha20'] + \
#                      data['alpha23'] + data['alpha36'] + data['alpha37'] + \
#                      data['alpha40'] + data['alpha41'] + data['alpha47'] + \
#                      data['alpha48'] + data['alpha50'])

# data['alpha_s2'] = -(data['alpha9'] + data['alpha16'] + data['alpha20'] + \
#                      data['alpha23'] + data['alpha47'] + \
#                      data['alpha48'])


# data['alpha_k3'] = data['alpha_k1'] + data['alpha_k2'] + data['alpha44'] + data['alpha51']
data['alpha_k4'] = data['alpha12'] + data['alpha13'] + data['alpha25'] + data['alpha44'] 
data['alpha_k5'] = data['alpha_k4'] + data['alpha76']
data['alpha_k6'] = data['alpha_k4'] + data['alpha59'] + data['alpha76'] 

# - 'alpha65', 'alpha71', 'alpha72', 'alpha74', 'alpha79', 'alpha84'


data['alpha_s3'] = -(data['alpha65'] + data['alpha71'] + data['alpha72'] + data['alpha84'])

#78in 46out


# data['final'] = data['alpha4'] + data['alpha52'] + data['alpha_k6'] + data['alpha54'] + data['alpha78'] + data['alpha_s3']


data['d1'] = 0
data['d2'] = 0
data['d3'] = 0

N1 = 10
N2 = 50
data.loc[(data['guidance_days_prev'] < N1), 'd1'] = 1.0
data.loc[(data['guidance_days_prev'] > N1) & (data['guidance_days_prev'] < N2), 'd2'] = 1.0
data.loc[data['guidance_days_prev'] > N2, 'd3'] = 1.0

data['alpha4_d1'] = data['alpha4']*data['d1']
data['alpha4_d2'] = data['alpha4']*data['d2']
data['alpha4_d3'] = data['alpha4']*data['d3']

data['alpha43_d1'] = data['alpha43']*data['d1']
data['alpha43_d2'] = data['alpha43']*data['d2']
data['alpha43_d3'] = data['alpha43']*data['d3']

data['alpha46_d1'] = data['alpha46']*data['d1']
data['alpha46_d2'] = data['alpha46']*data['d2']
data['alpha46_d3'] = data['alpha46']*data['d3']

data['alpha52_d1'] = data['alpha52']*data['d1']
data['alpha52_d2'] = data['alpha52']*data['d2']
data['alpha52_d3'] = data['alpha52']*data['d3']

data['alpha_k6_d1'] = data['alpha_k6']*data['d1']
data['alpha_k6_d2'] = data['alpha_k6']*data['d2']
data['alpha_k6_d3'] = data['alpha_k6']*data['d3']

data['alpha54_d1'] = data['alpha54']*data['d1']
data['alpha54_d2'] = data['alpha54']*data['d2']
data['alpha54_d3'] = data['alpha54']*data['d3']

data['alpha78_d1'] = data['alpha78']*data['d1']
data['alpha78_d2'] = data['alpha78']*data['d2']
data['alpha78_d3'] = data['alpha78']*data['d3']

data['alpha_s3_d1'] = data['alpha_s3']*data['d1']
data['alpha_s3_d2'] = data['alpha_s3']*data['d2']
data['alpha_s3_d3'] = data['alpha_s3']*data['d3']


significant_factor_names = [
                            'alpha4',
#                              'alpha43',
#                             'alpha46',
#                             'alpha52',
                            'alpha_k6',
#                             'alpha54',
                            'alpha78',
                            'alpha_s3',
                            'alpha86'
                           ]

# significant_factor_names = [
#                             'alpha4_d2',
#                             'alpha52_d2',
#                             'alpha_k6_d2',
#                             'alpha54_d2',
#                             'alpha78_d2',
#                             'alpha_s3_d2'
#                            ]


# significant_factor_names = [
# #                             'alpha4_d1',
#                             'alpha52_d1',
#                             'alpha_k6_d1',
#                             'alpha54_d1',
#                             'alpha78_d1',
#                             'alpha_s3_d1'
#                            ]


est = sm.OLS(
    100*data[['returns']], 
        data[significant_factor_names])

est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

In [ ]:
significant_factor_names = [
                            'alpha4_d2',
#                             'alpha52_d2',
                            'alpha_k6_d2',
#                             'alpha54_d2',
#                             'alpha78_d2',
#                             'alpha_s3_d2'
                           ]




est = sm.OLS(
    100*data[['returns']], 
        data[significant_factor_names])

est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

In [ ]:
significant_factor_names = [
                            'alpha4_d3',
                            'alpha52_d3',
                            'alpha_k6_d3',
                            'alpha54_d3',
                            'alpha78_d3',
                            'alpha_s3_d3'
                           ]




est = sm.OLS(
    100*data[['returns']], 
        data[significant_factor_names])

est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

In [ ]:

data['alpha_s3_d4'] = data['alpha_s3_d2'] + data['alpha_s3_d3'] 
data['alpha52_d4'] = data['alpha52_d2'] + data['alpha52_d3'] 
significant_factor_names = [
#     'alpha_k6',
#                             'alpha4_d1',
#                             'alpha52_d1',
#                             'alpha_k6_d1',
#                             'alpha54_d1',
#                             'alpha78_d1',
#                             'alpha_s3_d1',

    'alpha4_d2',
#                             'alpha52_d2',
                            'alpha_k6',
#                             'alpha54_d2',
#                             'alpha78_d2',
#                             'alpha_s3_d2',
#     'alpha4_d3',
#                             'alpha52_d3',
#                             'alpha_k6_d3',
#                             'alpha54_d3',
#                             'alpha78_d3',
#                             'alpha_s3_d3',
#     'alpha_s3_d4',
#         'alpha52_d4',
]




est = sm.OLS(
    100*data[['returns']], 
        data[significant_factor_names])

est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

In [ ]:
print data['alpha4'].corr(data['alpha4_d1'] + data['alpha4_d2'] + data['alpha4_d3'])

In [ ]:
d = data.set_index(['dates', 'sid'])
net_alpha = d['alpha_k6'] + d['alpha54_d1'] + d['alpha4_d2'] + d['alpha54_d3'] + d['alpha78_d3'] + d['alpha_s3_d4'] + d['alpha52_d4']
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
# alphalens.tears.create_summary_tear_sheet(factor_data)

In [ ]:
d = data.set_index(['dates', 'sid'])
net_alpha = d['alpha4'] + d['alpha52'] + d['alpha_k6'] + d['alpha54'] + d['alpha78'] + d['alpha_s3']
# net_alpha = d['alpha2'] + d['alpha4'] + d['alpha14'] + \
# d['alpha31'] + d['alpha35'] + d['alpha43'] + d['alpha45'] + \
# d['alpha46'] + d['alpha52'] + d['alpha55'] + d['alpha66'] + d['alpha73'] 
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
# alphalens.tears.create_summary_tear_sheet(factor_data)

In [ ]:
d = data.set_index(['dates', 'sid'])
net_alpha = d['alpha4'] + d['alpha52_d2'] + d['alpha_k6'] + d['alpha54'] + d['alpha78_d2'] + d['alpha_s3_d2']
# net_alpha = d['alpha2'] + d['alpha4'] + d['alpha14'] + \
# d['alpha31'] + d['alpha35'] + d['alpha43'] + d['alpha45'] + \
# d['alpha46'] + d['alpha52'] + d['alpha55'] + d['alpha66'] + d['alpha73'] 
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)

In [ ]:
d = data.set_index(['dates', 'sid'])
net_alpha = d['alpha4_d2'] + d['alpha52_d2'] + d['alpha_k6'] + d['alpha54'] + d['alpha78_d2'] + d['alpha_s3_d2']
# net_alpha = d['alpha2'] + d['alpha4'] + d['alpha14'] + \
# d['alpha31'] + d['alpha35'] + d['alpha43'] + d['alpha45'] + \
# d['alpha46'] + d['alpha52'] + d['alpha55'] + d['alpha66'] + d['alpha73'] 
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)

In [ ]:
d = data.set_index(['dates', 'sid'])
net_alpha = d['alpha_k6'] + d['alpha54'] + d['alpha78_d2'] + d['alpha_s3_d2'] + d['alpha52']
# net_alpha = d['alpha2'] + d['alpha4'] + d['alpha14'] + \
# d['alpha31'] + d['alpha35'] + d['alpha43'] + d['alpha45'] + \
# d['alpha46'] + d['alpha52'] + d['alpha55'] + d['alpha66'] + d['alpha73'] 
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)